In [1]:
from shared.preface import *
import shared.my_units as my
import shared.functions as fct
import shared.control_center as CC

## s_of_z with nb.jit

In [2]:
time_s = fct.s_of_z(1)
print(time_s)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1mUntyped global name 'quad':[0m [1m[1mCannot determine Numba type of <class 'function'>[0m
[1m
File "shared/functions.py", line 241:[0m
[1m    def s_integrand(z):
        <source elided>

[1m    s_of_z, _ = quad(s_integrand, 0., z)
[0m    [1m^[0m[0m
[0m

## Manual Conversion factors.

### -> solMass / kpc3

In [ ]:
z=3
H_squared = my.H0**2. * (my.Omega_m0*(1.+z)**3. + my.Omega_L0) 
rho_crit = 3.*H_squared / (8.*np.pi*const.G)

print(rho_crit)

in_units = rho_crit.to(unit.M_sun/unit.kpc**3.)
print(in_units)

cf = (rho_crit/in_units).value
print(cf)

print(rho_crit.value/cf)

### -> kpc / s2

In [ ]:
ratio1, ratio2, ratio3 = 3., 2., 1.
x_i = 8.5*unit.kpc 
r = 8.5*unit.kpc
prefactor = 4.*np.pi*const.G*my.rho0_NFW*(200*unit.kpc)**2.*np.abs(x_i)/r**2.
term1 = np.log(1.+ratio1) / ratio2
term2 = ratio3 / (1.+ratio1)
derivative_vector = prefactor * (term1 - term2)

print(derivative_vector)

in_rightUnits = derivative_vector.to(unit.kpc/unit.s**2.)
print(in_rightUnits)

cf2 = (derivative_vector/in_rightUnits).value

print(cf2)
print(derivative_vector.value/cf2)

In [ ]:
%%time

x1, x2, x3 = 8.5, 0., 0.
x0 = np.array([x1, x2, x3])
fct.dPsi_dxi_NFW(x0*unit.kpc, 1, my.rho0_NFW, my.Mvir_NFW)

In [ ]:
%%time


@nb.njit
def c_vir(z, M_vir):
    """Concentration parameter defined as r_vir/r_s, i.e. the ratio of virial 
    radius to the scale radius of the halo according to eqn. 5.5 of 
    Mertsch et al. (2020). 

    Args:
        z (array): redshift
        M_vir (float): virial mass, treated as fixed in time

    Returns:
        array: concentration parameters at each given redshift [dimensionless]
    """

    # Beta is then obtained from c_vir_avg(0, M_vir) and c_vir(0, M_vir).
    beta = (333.5/19.9) / fct.c_vir_avg(0, M_vir)

    c = beta * fct.c_vir_avg(z, M_vir)

    return c


zeds = np.linspace(1,100,10000)

sols = []
for z in zeds:
    c_val = c_vir(z, my.Mvir_NFW)
    sols.append(c_val)

print(len(sols), sols[0], sols[-1])